# Roocs data stores

Roocs consists of stores which it uses to identify and apply fixes to datasets. This notebook demonstrates the purpose of and how to interact with the Character store and the Fix store.

The Character store contains information about the 'character' of datasets that have been scanned. This is used in the identification of fixes.
    
The Fix store keeps track of these fixes. Each document corresponds to a specific dataset and details the fixes which are to be applied to it before an operation is applied to it.

In [ ]:
DIAGRAM

## Working with the character store

In [3]:
# Import the required packages
import virtualenv
import pip
import os

# Define and create the base directory install virtual environments
venvs_dir = os.path.join(os.path.expanduser("~"), "nb-venvs")

if not os.path.isdir(venvs_dir):
    os.makedirs(venvs_dir)
    
# Define the venv directory
venv_dir = os.path.join(venvs_dir, 'venv-dachar-notebook')

In [4]:
# Create the virtual environment
virtualenv.create_environment(venv_dir)

copying /opt/jaspy/bin/python => /home/users/esmith88/nb-venvs/venv-dachar-notebook/bin/python


In [5]:
# Activate the venv
activate_file = os.path.join(venv_dir, "bin", "activate_this.py")
exec(open(activate_file).read(), dict(__file__=activate_file))

In [15]:
# pip install a package using the venv as a prefix
pip.main(["install", "--prefix",venv_dir, "-e", "git+https://github.com/roocs/dachar.git@dev-elle#egg=dachar"])
# pip.main(["install", "--prefix", venv_dir, "git+https://github.com/roocs/dachar#egg=dachar"])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Obtaining dachar from git+https://github.com/roocs/dachar.git@dev-elle#egg=dachar


What to do?  (s)witch, (i)gnore, (w)ipe, (b)ackup  s


  Switching clone /home/users/esmith88/nb-venvs/venv-dachar-notebook/src/dachar to https://github.com/roocs/dachar.git (to revision dev-elle)


  Running command git config remote.origin.url https://github.com/roocs/dachar.git
  Running command git checkout -q dev-elle
  Running command git submodule update --init --recursive -q
    ERROR: Command errored out with exit status 1:
     command: /opt/jaspy/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/home/users/esmith88/nb-venvs/venv-dachar-notebook/src/dachar/setup.py'"'"'; __file__='"'"'/home/users/esmith88/nb-venvs/venv-dachar-notebook/src/dachar/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-fvawfp1p
         cwd: /home/users/esmith88/nb-venvs/venv-dachar-notebook/src/dachar/
    Complete output (1 lines):
    error in dachar setup command: 'install_requires' must be a string or list of strings containing valid project/version requirement specifiers; Invalid requirement, parse error at

1

In [11]:
from dachar.dachar.utils import get_stores
char_store = get_stores.get_dc_store('elasticsearch')
char_store.get_all()

ModuleNotFoundError: No module named 'dachar'